In [25]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [26]:
import folium
import pandas as pd

In [27]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [28]:
# 定義發射場的列表
launch_sites = [
    {"name": "Launch Site 1", "lat": 28.573255, "lon": -80.646895},
    {"name": "Launch Site 2", "lat": 29.559684, "lon": -95.083097},
    # 添加更多發射場
]

In [29]:
import folium

# 創建一個地圖
site_map = folium.Map(location=[28.573255, -80.646895], zoom_start=5)

# 為每個發射場添加標記
for site in launch_sites:
    folium.Marker(
        location=[site["lat"], site["lon"]],
        popup=site["name"],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(site_map)

# 顯示地圖
site_map

In [30]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [31]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [32]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [33]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [34]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

In [35]:
# 定義發射場及其啟動狀態
launch_sites = [
    {"name": "Launch Site 1", "lat": 28.573255, "lon": -80.646895, "status": "success"},
    {"name": "Launch Site 2", "lat": 29.559684, "lon": -95.083097, "status": "failure"},
    # 添加更多發射場
]

In [36]:
import folium

# 創建一個地圖
site_map = folium.Map(location=[28.573255, -80.646895], zoom_start=5)

# 為每個發射場添加標記
for site in launch_sites:
    # 根據啟動狀態選擇圖標顏色
    icon_color = 'green' if site["status"] == "success" else 'red'
    
    folium.Marker(
        location=[site["lat"], site["lon"]],
        popup=f"{site['name']} - {site['status']}",
        icon=folium.Icon(color=icon_color, icon='info-sign')
    ).add_to(site_map)

# 顯示地圖
site_map

In [37]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [38]:
marker_cluster = MarkerCluster()

In [39]:
# 假設 spacex_df 已經存在並且包含 'class' 列

# 定義一個函數來根據 class 值返回顏色
def get_marker_color(launch_class):
    if launch_class == 1:
        return 'green'  # 成功
    else:
        return 'red'    # 失敗

# 使用 apply 方法來建立 marker_color 列
spacex_df['marker_color'] = spacex_df['class'].apply(get_marker_color)

# 檢查結果
print(spacex_df[['class', 'marker_color']].tail(10))

    class marker_color
46      1        green
47      1        green
48      1        green
49      1        green
50      1        green
51      0          red
52      0          red
53      0          red
54      1        green
55      0          red


In [40]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

In [41]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [42]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [43]:
import math

def haversine(lat1, lon1, lat2, lon2):
    # 將緯度和經度從度轉換為弧度
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine 公式
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))

    # 地球半徑（公里）
    r = 6371

    # 計算結果
    return c * r

In [44]:
import folium

# 創建一個以特定位置為中心的地圖
site_map = folium.Map(location=[28.573255, -80.646895], zoom_start=10)

# 顯示地圖
site_map

In [45]:
import folium

# 創建一個以特定位置為中心的地圖
site_map = folium.Map(location=[28.573255, -80.646895], zoom_start=10)

# 顯示地圖
site_map
# 發射場的坐標
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# 最近海岸線的坐標
coastline_lat = 28.56367
coastline_lon = -80.57163

# 計算距離
distance_to_coastline = haversine(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

print(f"距離海岸線: {distance_to_coastline:.2f} 公里")

距離海岸線: 7.43 公里


In [46]:
# 發射場的坐標
launch_site_coord = (28.573255, -80.646895)

# 選定的海岸線點的坐標
coastline_point_coord = (28.56367, -80.57163)

In [47]:
import folium

# 創建一個以發射場為中心的地圖
site_map = folium.Map(location=launch_site_coord, zoom_start=10)

# 定義折線的坐標
coordinates = [launch_site_coord, coastline_point_coord]

# 創建折線
lines = folium.PolyLine(locations=coordinates, weight=1, color='blue')

# 將折線添加到地圖
site_map.add_child(lines)

# 顯示地圖
site_map

In [49]:
# 發射場的坐標
launch_site_coord = (28.573255, -80.646895)

# 定義最近城市、鐵路和高速公路的坐標
nearest_city_coord = (28.3852, -81.5639)  # 這是一個示例坐標
nearest_rail_coord = (28.5721, -80.5853)  # 這是一個示例坐標
nearest_highway_coord = (28.5721, -80.6500)  # 這是一個示例坐標

In [50]:
import folium
import math

def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371  # 地球半徑（公里）
    return c * r

# 創建地圖
site_map = folium.Map(location=launch_site_coord, zoom_start=10)

# 定義地點和顏色
locations = [
    ("Nearest City", nearest_city_coord, 'purple'),
    ("Nearest Rail", nearest_rail_coord, 'orange'),
    ("Nearest Highway", nearest_highway_coord, 'blue')
]

# 為每個地點計算距離並添加標記和折線
for name, coord, color in locations:
    distance = haversine(launch_site_coord[0], launch_site_coord[1], coord[0], coord[1])
    folium.Marker(
        location=coord,
        popup=f"{name}\nDistance: {distance:.2f} km",
        icon=folium.Icon(color=color, icon='info-sign')
    ).add_to(site_map)
    folium.PolyLine(locations=[launch_site_coord, coord], weight=1, color=color).add_to(site_map)

# 顯示地圖
site_map